In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("MiAplicacion") \
    .getOrCreate()

In [44]:
df = spark.read.parquet("data/final2/merge_jugadores2.parquet")
df.show(5)

+--------------------+-----------+---------+----------+--------+------+----------------+-------------+---------------+-----+-----------+---------+-----+--------------------+-----------------+---------------------------+---------------+---------------------+---------------------+----------+-----------------+------------------+------------+------------+------------+--------------+--------+----------------+--------+-------------+-------------+------+--------+------------+--------------+----------------------------+
|              nombre|club_actual|temporada|      liga|Posicion|nacido|partidos_jugados|titularidades|minutos_jugados|goles|asistencias|amarillas|rojas|carreras_progresivas|pases_progresivos|Pases_progresivos_recibidos|goles_esperados|asistencias_esperadas|goles_esperados_noPen|key_passes|          xGChain|         xGBuildup|entradas_1/3|entradas_2/3|entradas_3/3|duelos_ganados|entradas|entradas_ganadas|bloqueos|bloqueos_tiro|bloqueos_pase|altura|   valor|xG_xA_por_90|eficienci

df.columns

In [45]:
df3 = spark.read.parquet("data/final/merge_jugadores.parquet")
df3.show(5)

+--------------------+---------+--------------+--------+------+----------------+-------------+---------------+-----+-----------+---------+-----+--------------------+-----------------+---------------------------+---------------+---------------------+---------------------+----------+-----------------+------------------+------------+------------+------------+--------------+--------+----------------+--------+-------------+-------------+-----------+------+---------+------------+--------------+----------------------------+
|              nombre|temporada|          liga|Posicion|nacido|partidos_jugados|titularidades|minutos_jugados|goles|asistencias|amarillas|rojas|carreras_progresivas|pases_progresivos|Pases_progresivos_recibidos|goles_esperados|asistencias_esperadas|goles_esperados_noPen|key_passes|          xGChain|         xGBuildup|entradas_1/3|entradas_2/3|entradas_3/3|duelos_ganados|entradas|entradas_ganadas|bloqueos|bloqueos_tiro|bloqueos_pase|club_actual|altura|    valor|xG_xA_por_90

In [46]:
df_unidos = df.toPandas()
df_unidos.to_csv("unidos2.csv", index=False)

In [17]:
from pyspark.sql.functions import col
df1= df.filter(col("valor").isNull())
df1_pd = df1.toPandas()
df1_pd.to_csv("nounidos3.csv", index=False)

In [20]:
print(df1.filter(col("nombre") == "alex jimenez"))

DataFrame[nombre: string, temporada: string, liga: string, Posicion: string, nacido: string, partidos_jugados: string, titularidades: string, minutos_jugados: float, goles: float, asistencias: float, amarillas: string, rojas: string, carreras_progresivas: string, pases_progresivos: string, Pases_progresivos_recibidos: string, goles_esperados: float, asistencias_esperadas: float, goles_esperados_noPen: float, key_passes: string, xGChain: string, xGBuildup: string, entradas_1/3: string, entradas_2/3: string, entradas_3/3: string, duelos_ganados: string, entradas: string, entradas_ganadas: string, bloqueos: string, bloqueos_tiro: string, bloqueos_pase: string, club_actual: string, altura: string, valor: string, xG_xA_por_90: double, eficiencia_gol: double, asistencias_esperadas_por_90: double]


In [2]:

import os
import pandas as pd
from pathlib import Path
import unicodedata
import re
from rapidfuzz import process
df_fbref = pd.read_csv("data/limpios/fbref/laliga_2022_2023.csv")
df_mercado = pd.read_csv("data/limpios/mercado/valores_mercado_LaLiga_2022-2023.csv")



In [15]:
def mapear_nombres(lista_fuente, lista_objetivo, umbral=85):
    mapping = {}
    for nombre in lista_fuente:
        mejor, score, _ = process.extractOne(nombre, lista_objetivo)
        if score >= umbral:
            mapping[nombre] = mejor
    return mapping

# Ejemplo con understat y fbref
nombres_mercado = df_mercado["nombre"].unique().tolist()
nombres_fbref = df_fbref["nombre"].unique().tolist()

mapping = mapear_nombres(nombres_mercado, nombres_fbref)

# Reemplazar en understat los nombres
df_mercado["nombre"] = df_mercado["nombre"].replace(mapping)

df_mercado["nombre"].to_csv("prueba_data/mercado_nombres.csv", index=False)

In [3]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

In [66]:
df_fbref = pd.read_csv("data/limpios/fbref/fbref.csv")
df_fbref_defense = pd.read_csv("data/limpios/fbref_defense/fbref_defense.csv")
df_understat = pd.read_csv("data/limpios/understat/understat.csv")
df_mercado = pd.read_csv("data/limpios/mercado/mercado.csv")

fbref = df_fbref["club_actual"]
understat = df_understat["club_actual"]
mercado = df_mercado["club_actual"]

nombre_fbref = fbref.dropna().sort_values().unique()
nombre_understat = understat.dropna().sort_values().unique()
nombre_mercado = mercado.dropna().sort_values().unique()

pd.Series(nombre_fbref).to_csv("prueba_data/club_actual_fbref.csv", index=False)
pd.Series(nombre_understat).to_csv("prueba_data/club_actual_understat.csv", index=False)
pd.Series(nombre_mercado).to_csv("prueba_data/club_actual_mercado.csv", index=False)

In [31]:
duplicados = df_fbref[df_fbref.duplicated(subset=["nombre"], keep=False)]
print(duplicados)
df_ordenado = duplicados.sort_values(by="nombre")
df_ordenado.to_csv("prueba_data/duplicados_fbref.csv", index=False)

                nombre Posicion  nacido  partidos_jugados  titularidades  \
0      paxten aaronson    MF,DF  2003.0                 7              0   
1       sargis adamyan    FW,MF  1993.0                24              4   
2        karim adeyemi    FW,MF  2002.0                24             20   
3           amine adli    FW,MF  2000.0                26             16   
4            felix agu    DF,FW  1999.0                 3              0   
...                ...      ...     ...               ...            ...   
6399   piotr zielinski       MF  1994.0                26              8   
6400      nadir zortea    DF,MF  1999.0                35             33   
6401  szymon zurkowski       MF  1997.0                 5              0   
6402        milan uric       FW  1990.0                18             13   
6403        milan uric       FW  1990.0                 9              3   

      minutos_jugados  goles  asistencias  amarillas  rojas  \
0                 173   

In [ ]:

df_mercado = pd.read_csv("data/mercado/valores_mercado_LaLiga_2022-2023.csv")

def limpiar_ligas_transfermarkt(df):
    ligas_principales = ["Serie A", "LaLiga", "Premier League", "Bundesliga"]
    return df[df["comp_name"].isin(ligas_principales)]

df_mercado = limpiar_ligas_transfermarkt(df_mercado)

df_mercado.to_csv("prueba_data/mercadoLaLiga.csv", index=False)